# FEATURE SELECTION EDASPY

# INFO

El objetivo de este _notebook_ es la realización de _feature selection_ mediante algoritmos evolutivos.

## LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt # plots
import seaborn as sns # plots
from scipy import stats as sts # normality test JB
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.linear_model import Ridge, RidgeCV # Ridge
from sklearn.linear_model import Lasso, LassoCV # Lasso
from sklearn.linear_model import LogisticRegression # Log Regression
from sklearn.metrics import r2_score # R^2
from sklearn.metrics import mean_squared_error # MSE
from sklearn import preprocessing # estandarization
from sklearn.ensemble import IsolationForest # outliers
from math import sqrt # sqrt
import itertools # aplanar arrays
import math
from sklearn.svm import SVC # SVM
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import xgboost as xgb # XGBOOST
from sklearn.metrics import roc_curve, roc_auc_score, plot_roc_curve # ROC Curve
from sklearn.datasets import make_classification
import random
from sklearn.model_selection import train_test_split # split data
import pickle
import lightgbm as lgb
from EDAspy.optimization.univariate import EDA_discrete as EDAd
from sklearn.metrics import f1_score

C:\Users\migue\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## READING DATA

Carga y lectura de los datos con las transformaciones ya realizadas adecuadas a un modelo Lightgbm.

In [3]:
data_path = "../data/datos_procesados.csv"

In [4]:
datos = pd.read_csv(data_path)

In [5]:
datos

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307502,0,0,0,1,0,157500.0,254700.0,27558.0,225000.0,0.032561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307503,0,1,0,0,0,72000.0,269550.0,12001.5,225000.0,0.025164,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307504,0,1,0,0,0,153000.0,677664.0,29979.0,585000.0,0.005002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307505,1,1,0,0,0,171000.0,370107.0,20205.0,319500.0,0.005313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


We have:
* 307 511 entries.
* 122 variables:
    * 65 float.
    * 41 int.
    * 16 object, strings.

División en Train y Test

In [18]:
X = datos.loc[:, datos.columns != 'TARGET']
Y = datos['TARGET']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=32)

Se incluyen los hiperparámetros del modelo Lightgbm que obtuvo buenos resultados en la competición de Kaggle. La idea de comenzar con estos hiperparámetros es que se sabe que funcionan de forma adecuada y, por tanto, con los cambios que se han incluído, como por ejemplo asignar pesos distintos a cada una de las clases del TARGET, se podrán encontrar unos mejores hiperparámetros de forma más rápida.

In [21]:
modelo_final = lgb.LGBMClassifier(
            n_estimators=3000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            class_weight={0:1, 1:4.7})

Definición de la función de coste que se quiere maximizar con el uso de los algoritmos evolutivos EDAs.

In [22]:
def f1_gabri(y_test, y_pred):
    metric = (f1_score(y_test, y_pred, pos_label=0) * f1_score(y_test, y_pred, pos_label=1))**(1/2)
    return metric

In [24]:
def lgb_f1_score(y_pred, y_test):
    y_test = np.round(y_test) # scikits f1 doesn't like probabilities
    return 'f1_gabri', (f1_score(y_test, y_pred, pos_label=0) * f1_score(y_test, y_pred, pos_label=1))**(1/2), True

In [25]:
def costVarF1GeoF():
    def costVarF1Geo(dict_var):
        """
        - modelRB hace referencia al modelo Lightgbm que ha sido asignado anteriormente
        - para cada selección de diversas variables se hace un fit sobre la parte de train
        - se realiza una predicción para cada entrenamiento del modelo
        - se calcula y evalúa de la misma forma la función de coste que se busca maximizar
        
        """
        
        modelRB = modelo_final
        modelRB_feature_selection = [variable for variable in dict_var.keys() if dict_var[variable] == 1]
        modelRB.fit(X_train[modelRB_feature_selection], y_train,
                    eval_set=[(X_train[modelRB_feature_selection], y_train),
                              (X_test[modelRB_feature_selection], y_test)],
                    eval_metric= lgb_f1_score, verbose= 100, early_stopping_rounds= 150)
        resultsRB = modelRB.predict(X_test[modelRB_feature_selection])
        #f1g = f1_score(y_test, resultsRB)
        f1g = (f1_score(y_test, resultsRB, pos_label=0) * f1_score(y_test, resultsRB, pos_label=1))**(1/2)
        print(f1g)
        return f1g
    return costVarF1Geo

In [26]:
vector = pd.DataFrame(columns=X_train.columns.values)
vector.loc[0] = 0.5
costVarF1GeouEDF = costVarF1GeoF()
EDAuEDF = EDAd(SIZE_GEN=20, MAX_IT=10, DEAD_ITER=5, ALPHA=0.35, vector=vector,
            aim='maximize', cost_function=costVarF1GeouEDF)

- bestcost: muestra el valor más alto que se ha conseguido para la función de coste
- solution: refleja las variables seleccionadas con las que se ha conseguido el mayor valor para el f1 score geométrico
- history: muestra la evolución del valor de la función de coste que se buscaba maximizar a lo largo de las iteraciones

In [27]:
bestcost, solution, history = EDAuEDF.run(output=True)

Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.369195	valid_0's f1_gabri: 0.486542	valid_1's binary_logloss: 0.372165	valid_1's f1_gabri: 0.46427
[200]	valid_0's binary_logloss: 0.35434	valid_0's f1_gabri: 0.547469	valid_1's binary_logloss: 0.360021	valid_1's f1_gabri: 0.519949
[300]	valid_0's binary_logloss: 0.345938	valid_0's f1_gabri: 0.566111	valid_1's binary_logloss: 0.354403	valid_1's f1_gabri: 0.531218
[400]	valid_0's binary_logloss: 0.339824	valid_0's f1_gabri: 0.578812	valid_1's binary_logloss: 0.350895	valid_1's f1_gabri: 0.536995
[500]	valid_0's binary_logloss: 0.334542	valid_0's f1_gabri: 0.588027	valid_1's binary_logloss: 0.348144	valid_1's f1_gabri: 0.540024
[600]	valid_0's binary_logloss: 0.329893	valid_0's f1_gabri: 0.596624	valid_1's binary_logloss: 0.345844	valid_1's f1_gabri: 0.542431
[700]	valid_0's binary_logloss: 0.325832	valid_0's f1_gabri: 0.604708	valid_1's binary_logloss: 0.34407	valid_1's f1_gabri: 0.544156
[80

[900]	valid_0's binary_logloss: 0.321618	valid_0's f1_gabri: 0.616689	valid_1's binary_logloss: 0.344438	valid_1's f1_gabri: 0.5327
Early stopping, best iteration is:
[791]	valid_0's binary_logloss: 0.325776	valid_0's f1_gabri: 0.608242	valid_1's binary_logloss: 0.346136	valid_1's f1_gabri: 0.533679
0.533678588272024
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.36365	valid_0's f1_gabri: 0.517178	valid_1's binary_logloss: 0.3662	valid_1's f1_gabri: 0.495299
[200]	valid_0's binary_logloss: 0.348621	valid_0's f1_gabri: 0.562384	valid_1's binary_logloss: 0.353929	valid_1's f1_gabri: 0.533134
[300]	valid_0's binary_logloss: 0.340063	valid_0's f1_gabri: 0.57918	valid_1's binary_logloss: 0.348083	valid_1's f1_gabri: 0.546133
[400]	valid_0's binary_logloss: 0.33377	valid_0's f1_gabri: 0.588944	valid_1's binary_logloss: 0.344508	valid_1's f1_gabri: 0.551706
[500]	valid_0's binary_logloss: 0.32857	valid_0's f1_gabri: 0.598515	valid_1's binary_lo

Early stopping, best iteration is:
[661]	valid_0's binary_logloss: 0.331348	valid_0's f1_gabri: 0.597988	valid_1's binary_logloss: 0.347779	valid_1's f1_gabri: 0.534411
0.534410627168919
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.371743	valid_0's f1_gabri: 0.472792	valid_1's binary_logloss: 0.374302	valid_1's f1_gabri: 0.453709
[200]	valid_0's binary_logloss: 0.356648	valid_0's f1_gabri: 0.537384	valid_1's binary_logloss: 0.361871	valid_1's f1_gabri: 0.511668
[300]	valid_0's binary_logloss: 0.347942	valid_0's f1_gabri: 0.558155	valid_1's binary_logloss: 0.355773	valid_1's f1_gabri: 0.528739
[400]	valid_0's binary_logloss: 0.341723	valid_0's f1_gabri: 0.571846	valid_1's binary_logloss: 0.35215	valid_1's f1_gabri: 0.535713
[500]	valid_0's binary_logloss: 0.336605	valid_0's f1_gabri: 0.583287	valid_1's binary_logloss: 0.349471	valid_1's f1_gabri: 0.540492
[600]	valid_0's binary_logloss: 0.332091	valid_0's f1_gabri: 0.592702	valid_1's bi

0.555414038982567
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.371277	valid_0's f1_gabri: 0.474542	valid_1's binary_logloss: 0.373388	valid_1's f1_gabri: 0.454169
[200]	valid_0's binary_logloss: 0.357174	valid_0's f1_gabri: 0.539526	valid_1's binary_logloss: 0.362059	valid_1's f1_gabri: 0.516606
[300]	valid_0's binary_logloss: 0.348744	valid_0's f1_gabri: 0.559613	valid_1's binary_logloss: 0.356448	valid_1's f1_gabri: 0.528097
[400]	valid_0's binary_logloss: 0.342343	valid_0's f1_gabri: 0.573826	valid_1's binary_logloss: 0.352625	valid_1's f1_gabri: 0.535478
[500]	valid_0's binary_logloss: 0.337029	valid_0's f1_gabri: 0.584705	valid_1's binary_logloss: 0.349802	valid_1's f1_gabri: 0.537073
[600]	valid_0's binary_logloss: 0.332442	valid_0's f1_gabri: 0.594522	valid_1's binary_logloss: 0.34765	valid_1's f1_gabri: 0.53983
[700]	valid_0's binary_logloss: 0.328253	valid_0's f1_gabri: 0.602124	valid_1's binary_logloss: 0.345767	valid_1's f1_

Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.369586	valid_0's f1_gabri: 0.482995	valid_1's binary_logloss: 0.37232	valid_1's f1_gabri: 0.456611
[200]	valid_0's binary_logloss: 0.3538	valid_0's f1_gabri: 0.54706	valid_1's binary_logloss: 0.358919	valid_1's f1_gabri: 0.515029
[300]	valid_0's binary_logloss: 0.344816	valid_0's f1_gabri: 0.567251	valid_1's binary_logloss: 0.352542	valid_1's f1_gabri: 0.535403
[400]	valid_0's binary_logloss: 0.338401	valid_0's f1_gabri: 0.579469	valid_1's binary_logloss: 0.348614	valid_1's f1_gabri: 0.539542
[500]	valid_0's binary_logloss: 0.333288	valid_0's f1_gabri: 0.590884	valid_1's binary_logloss: 0.345861	valid_1's f1_gabri: 0.544197
[600]	valid_0's binary_logloss: 0.328862	valid_0's f1_gabri: 0.60002	valid_1's binary_logloss: 0.343806	valid_1's f1_gabri: 0.546027
[700]	valid_0's binary_logloss: 0.32484	valid_0's f1_gabri: 0.607562	valid_1's binary_logloss: 0.342129	valid_1's f1_gabri: 0.547213
[800]	

...

0.568463687661077
IT  8 best cost  0.5700718576046943
['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'AMT_INCOME_TOTAL', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_ID_PUBLISH', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'APARTMENTS_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'ELEVATORS_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAREA_MODE', 'APARTMENTS_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'TOTALAREA_MODE', 'OBS_60_CNT_SOCIAL_CIRCLE', 

Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.363671	valid_0's f1_gabri: 0.519197	valid_1's binary_logloss: 0.366084	valid_1's f1_gabri: 0.494125
[200]	valid_0's binary_logloss: 0.347985	valid_0's f1_gabri: 0.564328	valid_1's binary_logloss: 0.352986	valid_1's f1_gabri: 0.533253
[300]	valid_0's binary_logloss: 0.338922	valid_0's f1_gabri: 0.583332	valid_1's binary_logloss: 0.346524	valid_1's f1_gabri: 0.552774
[400]	valid_0's binary_logloss: 0.332399	valid_0's f1_gabri: 0.594079	valid_1's binary_logloss: 0.342531	valid_1's f1_gabri: 0.558167
[500]	valid_0's binary_logloss: 0.327009	valid_0's f1_gabri: 0.601799	valid_1's binary_logloss: 0.33972	valid_1's f1_gabri: 0.560607
[600]	valid_0's binary_logloss: 0.322328	valid_0's f1_gabri: 0.610297	valid_1's binary_logloss: 0.337609	valid_1's f1_gabri: 0.561605
[700]	valid_0's binary_logloss: 0.318005	valid_0's f1_gabri: 0.618613	valid_1's binary_logloss: 0.335754	valid_1's f1_gabri: 0.563282
[

[1100]	valid_0's binary_logloss: 0.303863	valid_0's f1_gabri: 0.64499	valid_1's binary_logloss: 0.330102	valid_1's f1_gabri: 0.567375
Early stopping, best iteration is:
[960]	valid_0's binary_logloss: 0.308808	valid_0's f1_gabri: 0.634693	valid_1's binary_logloss: 0.332123	valid_1's f1_gabri: 0.569493
0.5694926063889668
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.363611	valid_0's f1_gabri: 0.518465	valid_1's binary_logloss: 0.365947	valid_1's f1_gabri: 0.494933
[200]	valid_0's binary_logloss: 0.347966	valid_0's f1_gabri: 0.563489	valid_1's binary_logloss: 0.352752	valid_1's f1_gabri: 0.537949
[300]	valid_0's binary_logloss: 0.338888	valid_0's f1_gabri: 0.581669	valid_1's binary_logloss: 0.346386	valid_1's f1_gabri: 0.551199
[400]	valid_0's binary_logloss: 0.332283	valid_0's f1_gabri: 0.592497	valid_1's binary_logloss: 0.342287	valid_1's f1_gabri: 0.560694
[500]	valid_0's binary_logloss: 0.326882	valid_0's f1_gabri: 0.60141	valid_1's b

[700]	valid_0's binary_logloss: 0.318594	valid_0's f1_gabri: 0.616092	valid_1's binary_logloss: 0.335778	valid_1's f1_gabri: 0.565363
[800]	valid_0's binary_logloss: 0.314647	valid_0's f1_gabri: 0.623259	valid_1's binary_logloss: 0.334111	valid_1's f1_gabri: 0.565131
[900]	valid_0's binary_logloss: 0.310945	valid_0's f1_gabri: 0.629954	valid_1's binary_logloss: 0.332642	valid_1's f1_gabri: 0.566762
[1000]	valid_0's binary_logloss: 0.307305	valid_0's f1_gabri: 0.637002	valid_1's binary_logloss: 0.331211	valid_1's f1_gabri: 0.565589
Early stopping, best iteration is:
[892]	valid_0's binary_logloss: 0.311255	valid_0's f1_gabri: 0.62981	valid_1's binary_logloss: 0.332759	valid_1's f1_gabri: 0.567178
0.5671782088520807
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.363805	valid_0's f1_gabri: 0.517339	valid_1's binary_logloss: 0.366079	valid_1's f1_gabri: 0.49606
[200]	valid_0's binary_logloss: 0.34831	valid_0's f1_gabri: 0.563115	valid_1's bi

[1100]	valid_0's binary_logloss: 0.302654	valid_0's f1_gabri: 0.646661	valid_1's binary_logloss: 0.328807	valid_1's f1_gabri: 0.569389
[1200]	valid_0's binary_logloss: 0.299251	valid_0's f1_gabri: 0.653625	valid_1's binary_logloss: 0.327438	valid_1's f1_gabri: 0.569454
[1300]	valid_0's binary_logloss: 0.295823	valid_0's f1_gabri: 0.660787	valid_1's binary_logloss: 0.326114	valid_1's f1_gabri: 0.568342
Early stopping, best iteration is:
[1205]	valid_0's binary_logloss: 0.299071	valid_0's f1_gabri: 0.653904	valid_1's binary_logloss: 0.32736	valid_1's f1_gabri: 0.569704
0.5697035652780225
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.363701	valid_0's f1_gabri: 0.517709	valid_1's binary_logloss: 0.365991	valid_1's f1_gabri: 0.492789
[200]	valid_0's binary_logloss: 0.34812	valid_0's f1_gabri: 0.562591	valid_1's binary_logloss: 0.35291	valid_1's f1_gabri: 0.537923
[300]	valid_0's binary_logloss: 0.339136	valid_0's f1_gabri: 0.581454	valid_1's

In [28]:
bestcost

0.5700957342739739

In [29]:
solution

['CODE_GENDER',
 'AMT_INCOME_TOTAL',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'DAYS_BIRTH',
 'DAYS_ID_PUBLISH',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_EMAIL',
 'REGION_RATING_CLIENT',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'BASEMENTAREA_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'ELEVATORS_MODE',
 'FLOORSMAX_MODE',
 'LANDAREA_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_MEDI',
 'LIVINGAREA_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'TOTALAREA_MODE',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'FLAG_DO

In [30]:
history

[0.561557156261801,
 0.5603901243323224,
 0.5639236163865882,
 0.5695819717921313,
 0.5688336972899707,
 0.5688496086370187,
 0.5700957342739739,
 0.5695803492031657,
 0.5700718576046943,
 0.5699602891159371]

El algoritmo evolutivo a logrado un f1 score geométrico de 0.57 con 410 variables.

In [31]:
len(solution)

410

In [38]:
datos_edaspy = datos.loc[:, solution]
datos_edaspy

,CODE_GENDER,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_ID_PUBLISH,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_EMAIL,REGION_RATING_CLIENT,...,CC_NAME_CONTRACT_STATUS_Refused_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_MIN,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_SUM
0,0,202500.0,24700.5,351000.0,-9461,-2120,0,1,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,270000.0,35698.5,1129500.0,-16765,-291,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,67500.0,6750.0,135000.0,-19046,-2531,1,1,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,135000.0,29686.5,297000.0,-19005,-2437,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,121500.0,21865.5,513000.0,-19932,-3458,0,1,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307502,0,157500.0,27558.0,225000.0,-9327,-1982,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307503,1,72000.0,12001.5,225000.0,-20775,-4090,0,1,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307504,1,153000.0,29979.0,585000.0,-14966,-5150,0,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307505,1,171000.0,20205.0,319500.0,-11961,-931,0,1,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
datos_edaspy.to_csv("../data/datos_edaspy.csv", index = False)